In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

data = pd.read_csv('pb_data.csv') 

X = data.drop(columns=['ID', 'Bandgap'])
y = data['Bandgap']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=150, batch_size=8, validation_split=0.2, verbose=1)

test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print("Test Loss (MSE):", test_loss)
print("Test MAE:", test_mae)

predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print("R-squared:", r2)


In [ ]:
import plotly.graph_objects as go

x_test=X
print(x_test)

y_pred_nn = model.predict(x_test).flatten()

Cl_ratio = x_test['Cl'] / (x_test['Cl'] + x_test['Br'] + x_test['I'])
Br_ratio = x_test['Br'] / (x_test['Cl'] + x_test['Br'] + x_test['I'])
I_ratio = x_test['I'] / (x_test['Cl'] + x_test['Br'] + x_test['I'])

fig_a = go.Figure(data=[go.Scatter3d(
    x=Br_ratio,
    y=Cl_ratio,
    z=I_ratio,
    mode='markers',
    marker=dict(
        size=6,
        color=y_pred_nn,
        colorscale='turbo',
        cmin=0.7,
        colorbar=dict(
            title='Predicted Bandgap (eV)',
            #tickvals=[i for i in np.arange(1.4,3.0,0.2)],
            #ticktext=[str(i)for i in np.arange(1.4,3.0,0.2)]
        ),
        opacity=0.8
    )
)])

fig_a.update_layout(
    scene=dict(
        xaxis_title='Br/(Cl+Br+I)',
        yaxis_title='Cl/(Cl+Br+I)',
        zaxis_title='I/(Cl+Br+I)',
    ),
    title="4D Plot (a): Predicted Bandgap vs Halide Ratios",
    width=800,
    height=600
)

fig_a.show()


FA_ratio = x_test['FA'] / (x_test['FA'] + x_test['MA'] + x_test['Cs'])
MA_ratio = x_test['MA'] / (x_test['FA'] + x_test['MA'] + x_test['Cs'])
Cs_ratio = x_test['Cs'] / (x_test['FA'] + x_test['MA'] + x_test['Cs'])

fig_b = go.Figure(data=[go.Scatter3d(
    x=FA_ratio,
    y=MA_ratio,
    z=Cs_ratio,
    mode='markers',
    marker=dict(
        size=6,
        color=y_pred_nn,
        colorscale='turbo',
        cmin=1.2,
        cmax=1.8,
        colorbar=dict(
            title='Predicted Bandgap (eV)',
            #tickvals=[i for i in np.arange(0.7,1.7,0.1)],
            #ticktext=[str(i) for i in np.arange(0.7,1.7,0.1)]
        ),
        opacity=0.8
    )
)])

fig_b.update_layout(
    scene=dict(
        xaxis_title='FA/(FA+MA+Cs)',
        yaxis_title='MA/(FA+MA+Cs)',
        zaxis_title='Cs/(FA+MA+Cs)',
    ),
    title="4D Plot (b): Predicted Bandgap vs Cation Ratios",
    width=800,
    height=600
)

fig_b.show()

In [ ]:
import matplotlib.pyplot as plt
y_pred_nn = model.predict(X_test).flatten()
plt.scatter(y_test, y_pred_nn, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Bandgap')
plt.ylabel('Predicted Bandgap')
plt.title('Simple Neural Network: Actual vs Predicted Bandgap')